In [1]:
# Importy do całego projektu
import os
from IPython.display import display, clear_output
from ultralytics import YOLO
from pathlib import Path
import cv2
import numpy as np
import random
from tqdm import tqdm
import shutil

In [2]:
import torch
torch.cuda.is_available()

True

In [10]:
import kagglehub
dataset_path1 = kagglehub.dataset_download("jessicali9530/lfw-dataset")
dataset_path2 = kagglehub.dataset_download("chiragsaipanuganti/morph")
dataset_path3 = kagglehub.dataset_download("tapakah68/facial-emotion-recognition")
dataset_path4 = kagglehub.dataset_download("atulanandjha/lfwpeople")
dataset_path5 = kagglehub.dataset_download("ashishjangra27/gender-recognition-200k-images-celeba")
dataset_path6 = kagglehub.dataset_download("trainingdatapro/age-detection-human-faces-18-60-years")

100%|██████████| 1.32G/1.32G [12:58<00:00, 1.82MB/s]

Extracting files...


100%|██████████| 337M/337M [03:55<00:00, 1.50MB/s] 

Extracting files...


In [11]:
# Pobranie ścieżek do obrazów
image_paths = []
datasets = [dataset_path1, dataset_path2, dataset_path3, 
			dataset_path4, dataset_path5, dataset_path6]
for dataset_path in datasets:
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.lower().endswith(('.jpg')):
                image_paths.append(os.path.join(root, file))

In [17]:
def populate_card_labels_v3(image_path, im_width=250):
	img = cv2.imread(image_path)
	if img is None:
		return None, None
	height, width = img.shape[:2]
	im_height = int(height * im_width / width)
	img = cv2.resize(img, (im_width, im_height))
	img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

	diagonal_size = int(np.sqrt(im_width**2 + im_height**2)) + 10
	rot_canvas = np.ones((diagonal_size, diagonal_size, 3), dtype=np.uint8) * 255
	start_x = (diagonal_size - im_width) // 2
	start_y = (diagonal_size - im_height) // 2
	rot_canvas[start_y:start_y + im_height, start_x:start_x + im_width] = img

	rot_angle = random.uniform(-179, 180)
	center = (diagonal_size // 2, diagonal_size // 2)
	rot_matrix = cv2.getRotationMatrix2D(center, rot_angle, 1.0)
	rot_img = cv2.warpAffine(rot_canvas, rot_matrix, (diagonal_size, diagonal_size), borderValue=(255, 255, 255))

	height, width = rot_img.shape[:2]
	card = np.ones((height+10, width+10, 3), dtype=np.uint8) * 255
	card[5:5+height, 5:5+width] = rot_img

	corners = np.array([
		[start_x, start_y],
		[start_x + im_width, start_y],
		[start_x + im_width, start_y + im_height],
		[start_x, start_y + im_height]
	], dtype=np.float32)

	ones = np.ones((corners.shape[0], 1))
	corners_homo = np.hstack([corners, ones])
	rotated_corners = (rot_matrix @ corners_homo.T).T
	rotated_corners += np.array([5, 5])

	x_min, y_min = rotated_corners.min(axis=0)
	x_max, y_max = rotated_corners.max(axis=0)

	bbox_cx = (x_min + x_max) / 2 / (width+10)
	bbox_cy = (y_min + y_max) / 2 / (height+10)
	bbox_w = (x_max - x_min) / (width+10)
	bbox_h = (y_max - y_min) / (height+10)

	norm_corners = []
	for x, y in rotated_corners:
		norm_corners.extend([x / (width+10), y / (height+10), 2])

	label = [0, bbox_cx, bbox_cy, bbox_w, bbox_h] + norm_corners

	return card, label

def get_cards_labels(image_paths, num, im_width, output_path, starting_idx=0):
	output_path = Path(output_path)
	images_path = "images" / output_path
	labels_path = "labels" / output_path
	images_path.mkdir(parents=True, exist_ok=True)
	labels_path.mkdir(parents=True, exist_ok=True)
	cards = []
	wrong = 0
    
	for i in range(starting_idx, starting_idx+num):
		card, label = populate_card_labels_v3(
			image_paths[i],
			im_width=im_width
		)
		if card is None or label is None:
			wrong += 1
			continue

		filename = f"card_{i+1}"
		image_file = images_path / f"{filename}.jpg"
		label_file = labels_path / f"{filename}.txt"
        
		card_bgr = cv2.cvtColor(card, cv2.COLOR_RGB2BGR)
		cv2.imwrite(str(image_file), card_bgr)
        
		label_line = " ".join(f"{v:.6f}" if isinstance(v, float) else str(v) for v in label)

		with open(label_file, "w") as f:
			f.write(label_line)

		cards.append({"filename": f"{filename}.jpg", "label": label})
    
	if wrong > 0:
		print(f"Skipped {wrong} images due to errors.")
	return cards

In [18]:
shutil.rmtree("images", ignore_errors=True)
shutil.rmtree("labels", ignore_errors=True)

random.shuffle(image_paths)

i = 0
max = len(image_paths)

train_p = int(max * 0.9)
valid_p = int(max * 0.05)
test_p = max - train_p - valid_p

res = [250, 300, 150, 400, 500]
res_n = int(train_p / len(res))
res_idx = 0-res_n

iter = len(res)
progress = None
progress = tqdm(total=3*len(res), desc="Generating cards", unit="card")
for j in res:
	res_idx += res_n
	cards = get_cards_labels(image_paths, num=res_n, im_width=j, output_path='train', starting_idx=res_idx)
	progress.update(1)
res_n = int(valid_p / len(res))
for j in res:
	res_idx += res_n
	cards = get_cards_labels(image_paths, num=res_n, im_width=j, output_path='valid', starting_idx=res_idx)
	progress.update(1)
res_n = int(test_p / len(res))
for j in res:
	res_idx += res_n
	cards = get_cards_labels(image_paths, num=res_n, im_width=j, output_path='test', starting_idx=res_idx)
	progress.update(1)

if progress:
	progress.close()

Generating cards: 100%|██████████| 15/15 [10:43<00:00, 42.90s/card]


In [46]:
model = YOLO("runs/pose/image_rotation_model_train/weights/best.pt")

In [19]:
model = YOLO("yolo11x-pose.pt")

In [20]:
model.train(
    data='image_rotation.yaml',
    epochs=10,
    imgsz=416,
    batch=32,
    degrees=5.0,
    mosaic=0.0,
    auto_augment='none',
    erasing=0.0,
    cache='ram',
    name='image_rotation_model_anew',
    exist_ok=True,
)

New https://pypi.org/project/ultralytics/8.3.152 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.144 🚀 Python-3.11.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=none, batch=32, bgr=0.0, box=7.5, cache=ram, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=image_rotation.yaml, degrees=5.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.0, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11x-pose.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=image_rotation_model_anew, nbs=64, nms=Fal

train: Scanning /repos/image-corner-rotation/labels/train... 6859 images, 0 backgrounds, 0 corrupt:   3%|▎         | 6859/251440 [00:06<04:02, 1006.53it/s]


KeyboardInterrupt: 

In [35]:
model = YOLO('runs/pose/image_rotation_model_train/weights/best.pt')

results = model.predict(
    source='images/test',
    save=True,
    conf=0.25,
    imgsz=416
)


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1320 /repos/image-corner-rotation/images/test/card_11906.jpg: 416x416 1 rotated_photo, 14.8ms
image 2/1320 /repos/image-corner-rotation/images/test/card_11907.jpg: 416x416 1 rotated_photo, 31.7ms
image 3/1320 /repos/image-corner-rotation/images/test/card_11908.jpg: 416x416 1 rotated_photo, 21.0ms
image 4/1320 /repos/image-corner-rotation/images/test/card_11909.jpg: 416x416 1 rotated_photo, 15.2ms
image 5/1320 /repos/image-corner-rotation/images

In [ ]:
import time
import os
import subprocess
from ultralytics.utils import callbacks

README_FILE_PATH = "README.md"
REPO_ID = "Qbsoon/image-corner-rotation"
UPDATE_FREQUENCY = 'epoch'
BATCH_UPDATE_INTERVAL = 50
TIME_UPDATE_INTERVAL_SECONDS = 300
LAST_UPDATE_TIME_FOR_README = time.time()

status_data = {
    "attempt": 1,
    "modelName": "YourModelName",
    "startDate": "N/A",
    "lastUpdate": "N/A",
    "datasetSize": "251440 Images",
    "equipment": "1x RTX 4070Ti Super (16GB VRAM)",
    "estimatedTotalTime": "~N/A",
    "currentEpoch": 0,
    "totalEpochs": 0,
    "overallProgress": 0.0,
    "statusText": "Initializing...",
    "training_active": False
}

def generate_widget_html(current_status):
    spinner_animation_style = "animation: spin 1s linear infinite;" if current_status["training_active"] else "display: none;"
    display_progress = min(current_status["overallProgress"], 100.0)

    start_date_str = current_status["startDate"]
    last_update_str = current_status["lastUpdate"]
    if isinstance(start_date_str, float): start_date_str = time.strftime("%b %d, %Y", time.localtime(start_date_str))
    if isinstance(last_update_str, float): last_update_str = time.strftime("%b %d, %Y %H:%M:%S", time.localtime(last_update_str))


    html = f"""
<div class="card">
    <div class="header">
      <div class="spinner" style="{spinner_animation_style}"></div>
      <h2>{current_status["modelName"]} training</h2>
    </div>
    <div class="info-blocks">
      <div class="info-block"><strong>START DATE:</strong> {start_date_str}</div>
      <div class="info-block"><strong>LAST UPDATE:</strong> {last_update_str}</div>
      <div class="info-block"><strong>DATASET SIZE:</strong> {current_status["datasetSize"]}</div>
      <div class="info-block"><strong>EQUIPMENT:</strong> {current_status["equipment"]}</div>
      <div class="info-block"><strong>TRAINING TIME:</strong> {current_status["estimatedTotalTime"]}</div>
    </div>
    <div class="progress-container">
      <div class="progress-bar" style="width: {display_progress}%;"></div>
    </div>
    <div class="status-text">{current_status["statusText"]}</div>
</div>
"""
    return html

def update_readme_and_upload_with_hf_cli(widget_html_content):
    global REPO_ID
    if REPO_ID == "YOUR_USERNAME/YOUR_MODEL_NAME":
        print("ERROR: Please update REPO_ID in the script with your Hugging Face repository ID.")
        return

    try:
        with open(README_FILE_PATH, "r", encoding="utf-8") as f:
            readme_content = f.read()

        start_placeholder = "<!-- WIDGET_START -->"
        end_placeholder = "<!-- WIDGET_END -->"
        
        start_index = readme_content.find(start_placeholder)
        end_index = readme_content.find(end_placeholder)

        if start_index != -1 and end_index != -1:
            pre_widget = readme_content[:start_index + len(start_placeholder)]
            post_widget = readme_content[end_index:]
            new_readme_content = pre_widget + "\n" + widget_html_content + "\n" + post_widget
        else:
            print(f"Warning: Widget placeholders not found in {README_FILE_PATH}. Creating new content with widget.")
            # Updated CSS here
            html_template_start = """
<html>
<head>
  <style>
    /* Paste your full CSS styles here */
    body {{ font-family: Arial, sans-serif; }} /* Note: Double curly braces for f-string like formatting if this were an f-string, but it's a direct string here so single is fine. Keeping double for safety if it's ever converted. */
    .card {{ background: #FF8C00; border-radius: 16px; padding: 24px; width: 100%; max-width: 700px; box-shadow: 0 6px 12px rgba(0,0,0,0.2); color: #fff; box-sizing: border-box; margin: 20px auto; }}
    .card .header {{ display: flex; align-items: center; margin-bottom: 16px; }}
    .card .spinner {{ width: 24px; height: 24px; margin-right: 12px; border: 4px solid rgba(255,255,255,0.5); border-top-color: #fff; border-radius: 50%; animation: spin 1s linear infinite; }}
    .card .header h2 {{ font-size: 1.2rem; margin: 0; }}
    .card .info-blocks {{ display: flex; flex-wrap: nowrap; border: 1px solid rgba(255,255,255,0.5); border-radius: 12px; overflow: hidden; margin-bottom: 16px; }}
    .card .info-block {{ flex-grow: 1; flex-shrink: 1; flex-basis: 0; min-width: 0; padding: 8px 12px; font-size: 0.85rem; text-align: center; box-sizing: border-box; }}
    .card .info-block + .info-block {{ border-left: 1px solid rgba(255,255,255,0.5); }}
    .card .info-block strong {{ display: block; margin-bottom: 4px; font-weight: 600; text-transform: uppercase; }}
    .card .progress-container {{ background: rgba(255,255,255,0.3); border-radius: 12px; overflow: hidden; height: 14px; margin-bottom: 8px; }}
    .card .progress-bar {{ width: 0%; height: 100%; background: #fff; transition: width 0.5s ease-out; }}
    .card .status-text {{ font-size: 0.9rem; text-align: center; }}
    @keyframes spin {{ to {{ transform: rotate(360deg); }} }}
  </style>
</head>
<body>
{widget_content}
</body>
</html>
"""
            new_readme_content = html_template_start.format(widget_content=start_placeholder + "\n" + widget_html_content + "\n" + end_placeholder)


        with open(README_FILE_PATH, "w", encoding="utf-8") as f:
            f.write(new_readme_content)

        commit_message = f"Update training progress: {status_data.get('statusText', 'Progress update')}"
        subprocess.run([
            "huggingface-cli", "upload",
            REPO_ID,
            README_FILE_PATH,
            "README.md",
            "--commit-message", commit_message
        ], check=True, capture_output=True, text=True)
        print(f"README.md uploaded to {REPO_ID} at {time.strftime('%Y-%m-%d %H:%M:%S')}")

    except FileNotFoundError:
        print(f"Error: {README_FILE_PATH} not found. Ensure it exists in the current directory: {os.getcwd()}")
    except subprocess.CalledProcessError as e:
        print(f"Error during huggingface-cli upload: {e}")
        print(f"stdout: {e.stdout}")
        print(f"stderr: {e.stderr}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def on_train_start_callback(trainer):
    global LAST_UPDATE_TIME_FOR_README, status_data
    status_data["training_active"] = True
    status_data["totalEpochs"] = trainer.epochs
    status_data["startDate"] = time.time()
    status_data["lastUpdate"] = time.time()
    status_data["statusText"] = "Training started..."
    status_data["overallProgress"] = 0.0
    status_data["currentEpoch"] = 0
    widget_html = generate_widget_html(status_data)
    update_readme_and_upload_with_hf_cli(widget_html)
    LAST_UPDATE_TIME_FOR_README = time.time()

def on_epoch_end_callback(trainer):
    global LAST_UPDATE_TIME_FOR_README, status_data
    status_data["currentEpoch"] = trainer.epoch + 1
    current_epoch_progress = 1.0
    overall_progress_val = ((trainer.epoch + current_epoch_progress) / trainer.epochs) * 100
    status_data["overallProgress"] = round(overall_progress_val, 2)
    status_data["statusText"] = f"Fine-Tuning... {status_data['overallProgress']}% (Epoch {status_data['currentEpoch']}/{status_data['totalEpochs']})"
    status_data["lastUpdate"] = time.time()
    
    if UPDATE_FREQUENCY == 'epoch':
        widget_html = generate_widget_html(status_data)
        update_readme_and_upload_with_hf_cli(widget_html)
        LAST_UPDATE_TIME_FOR_README = time.time()

def on_batch_end_callback(trainer):
    global LAST_UPDATE_TIME_FOR_README, status_data

    if UPDATE_FREQUENCY == 'batch' and (trainer.batch_idx + 1) % BATCH_UPDATE_INTERVAL == 0:
        current_epoch_progress = (trainer.batch_idx + 1) / len(trainer.train_loader)
        overall_progress_val = ((trainer.epoch + current_epoch_progress) / trainer.epochs) * 100
        status_data["overallProgress"] = round(overall_progress_val, 2)
        status_data["statusText"] = f"Fine-Tuning... {status_data['overallProgress']}% (Epoch {trainer.epoch + 1}/{trainer.epochs})"
        status_data["lastUpdate"] = time.time()
        widget_html = generate_widget_html(status_data)
        update_readme_and_upload_with_hf_cli(widget_html)
        LAST_UPDATE_TIME_FOR_README = time.time()

def on_train_end_callback(trainer):
    global status_data
    status_data["training_active"] = False
    status_data["overallProgress"] = 100.0
    status_data["statusText"] = "Training completed!"
    status_data["lastUpdate"] = time.time()
    widget_html = generate_widget_html(status_data)
    update_readme_and_upload_with_hf_cli(widget_html)
    print("Final README.md update uploaded.")


if hasattr(model, 'callbacks'):
    model.callbacks = {}
    
model.add_callback("on_train_start", on_train_start_callback)
model.add_callback("on_epoch_end", on_epoch_end_callback)
if UPDATE_FREQUENCY == 'batch':
    model.add_callback("on_batch_end", on_batch_end_callback)
model.add_callback("on_train_end", on_train_end_callback)